In [1]:
import xarray as xr
import pandas as pd
import reverse_geocode
import sqlite3

In [2]:
# Define file paths
file_path_1 ="data_1.nc"
file_path_0 = "data_0.nc"

In [3]:
# Open the NetCDF files
dataset_1 = xr.open_dataset(file_path_1, engine='netcdf4')
dataset_0 = xr.open_dataset(file_path_0, engine='netcdf4')
print(dataset_0.data_vars.keys())
print(dataset_1.data_vars.keys())

KeysView(Data variables:
    cvh      (valid_time, latitude, longitude) float32 ...
    sp       (valid_time, latitude, longitude) float32 ...
    tco3     (valid_time, latitude, longitude) float32 ...
    si10     (valid_time, latitude, longitude) float32 ...
    skt      (valid_time, latitude, longitude) float32 ...)
KeysView(Data variables:
    uvb         (valid_time, latitude, longitude) float32 ...
    sf          (valid_time, latitude, longitude) float32 ...
    ttr         (valid_time, latitude, longitude) float32 ...
    tp          (valid_time, latitude, longitude) float32 ...
    crr         (valid_time, latitude, longitude) float32 ...
    avg_ie      (valid_time, latitude, longitude) float32 ...
    avg_vimdf   (valid_time, latitude, longitude) float32 ...
    avg_tprate  (valid_time, latitude, longitude) float32 ...)


In [4]:
df1 = dataset_1.to_dataframe().reset_index()

In [5]:
df0 = dataset_0.to_dataframe().reset_index()

In [6]:

    df0.rename(columns={'valid_time': 'date','cvh':'high_vegetation_cover','sp':'surface_pressure','tco3':'total_ozone','si10':'wind_speed','skt':'skin_temperature'},inplace=True)
    df1.rename(columns={'valid_time': 'date','uvb':'uv_radiation','sf':'snowfall','ttr':'net_thermal_radiation','tp':'total_precipitation','crr':'convective_rain_rate','avg_ie':'mean_evaporation_rate','avg_vimdf':'mean_moisture_divergence','avg_tprate':'mean_precipitation_rate'}, inplace=True)

In [7]:
#Function for Reverse Geocoding
def get_place_name(lat, lon):
    location = reverse_geocode.search([(lat, lon)])[0]
    city, country = location['city'], location['country']
    return city, country

In [8]:
def data_filter(ds,latmin,latmax,lonmin,lonmax):
    climate_subset = ds.query(f"latitude >= {latmin} and latitude <= {latmax} and longitude >= {lonmin} and longitude <= {lonmax}")
    return climate_subset

In [9]:
def final_df(mdf,cname):
    mdf[['City', 'Country']] = mdf.apply(lambda row: pd.Series(get_place_name(row["latitude"], row["longitude"])), axis=1)
    mdf = mdf[mdf["Country"] == cname]
    mdf = mdf.groupby(["City", "date"]).mean(numeric_only=True).reset_index()
    mdf = mdf.drop(columns={'number'})
    return mdf

In [10]:
india_df0=data_filter(df0,6.5,37.5,68.0,97.5)
nepal_df0 = final_df(india_df0,"Nepal")
india_df1=data_filter(df1,6.5,37.5,68.0,97.5)
nepal_df1 = final_df(india_df1,"Nepal")


/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
print(nepal_df1.columns)

Index(['City', 'date', 'latitude', 'longitude', 'uv_radiation', 'snowfall',
       'net_thermal_radiation', 'total_precipitation', 'convective_rain_rate',
       'mean_evaporation_rate', 'mean_moisture_divergence',
       'mean_precipitation_rate'],
      dtype='object')


In [13]:
india_df0=data_filter(df0,6.5,37.5,68.0,97.5)
bhutan_df0 = final_df(india_df0,"Bhutan")
india_df1=data_filter(df1,6.5,37.5,68.0,97.5)
bhutan_df1 = final_df(india_df1,"Bhutan")


In [14]:
india_df0=data_filter(df0,6.5,37.5,68.0,97.5)
india_df0 = final_df(india_df0,"India")
india_df1=data_filter(df1,6.5,37.5,68.0,97.5)
india_df1=final_df(india_df1,"India")


In [15]:
bangladesh_df0=data_filter(df0,20.57,26.63,88.02,92.68)
bangladesh_df0 = final_df(bangladesh_df0,"Bangladesh")
bangladesh_df1=data_filter(df1,20.57,26.63,88.02,92.68)
bangladesh_df1 = final_df(bangladesh_df1,"Bangladesh")

In [16]:
pakistan_df0=data_filter(df0,23.63,37.08,60.87,77.84)
pakistan_df0=final_df(pakistan_df0,"Pakistan")
pakistan_df1=data_filter(df1,23.63,37.08,60.87,77.84)
pakistan_df1=final_df(pakistan_df1,"Pakistan")

In [17]:
srilanka_df0=data_filter(df0,5.92,9.84,79.65,81.88)
srilanka_df0=final_df(srilanka_df0,"Sri Lanka")
srilanka_df1=data_filter(df1,5.92,9.84,79.65,81.88)
srilanka_df1=final_df(srilanka_df1,"Sri Lanka")

In [18]:
afghanistan_df0=data_filter(df0,29.38,38.49,60.53,74.89)
afghanistan_df0=final_df(afghanistan_df0,"Afghanistan")
afghanistan_df1=data_filter(df1,29.38,38.49,60.53,74.89)
afghanistan_df1=final_df(afghanistan_df1,"Afghanistan")

In [19]:
conn = sqlite3.connect('south_asia_monthly_new.db')

In [20]:
india_df0.to_sql('india_df0', conn, if_exists='replace', index=False)
nepal_df0.to_sql('nepal_df0', conn, if_exists='replace', index=False)
bhutan_df0.to_sql('bhutan_df0', conn, if_exists='replace', index=False)
pakistan_df0.to_sql('pakistan_df0',conn,if_exists='replace',index=False)
bangladesh_df0.to_sql('bangladesh_df0',conn,if_exists='replace',index=False)
srilanka_df0.to_sql('srilanka_df0',conn,if_exists='replace',index=False)
afghanistan_df0.to_sql('afghanistan_df0',conn,if_exists='replace',index=False)

In [21]:
india_df1.to_sql('india_df1', conn, if_exists='replace', index=False)
nepal_df1.to_sql('nepal_df1', conn, if_exists='replace', index=False)
bhutan_df1.to_sql('bhutan_df1', conn, if_exists='replace', index=False)
pakistan_df1.to_sql('pakistan_df1',conn,if_exists='replace',index=False)
bangladesh_df1.to_sql('bangladesh_df1',conn,if_exists='replace',index=False)
srilanka_df1.to_sql('srilanka_df1',conn,if_exists='replace',index=False)
afghanistan_df1.to_sql('afghanistan_df1',conn,if_exists='replace',index=False)


In [ ]:
print(list(india_df0["City"].unique()))


['7LC', 'Abhayāpuri', 'Abirāmam', 'Abohar', 'Abrama', 'Achalpur', 'Addanki', 'Adirampattinam', 'Adra', 'Aduthurai', 'Adūr', 'Afzalpur', 'Agar', 'Agartala', 'Agatti', 'Ahiri', 'Ahmadnagar', 'Ahmadpur', 'Ahraura', 'Ahwa', 'Aizawl', 'Ajani Khurd', 'Ajjampur', 'Ajmer', 'Ajra', 'Akalkot', 'Akaltara', 'Akasahebpet', 'Akbarpur', 'Aklera', 'Akola', 'Akot', 'Alagāpuram', 'Alampur', 'Aland', 'Alangāyam', 'Alappuzha', 'Allāhganj', 'Allāpalli', 'Almora', 'Alnāvar', 'Along', 'Alot', 'Alwar', 'Alībāg', 'Alīganj', 'Alīgarh', 'Alīpur Duār', 'Alūr', 'Amalner', 'Amalāpuram', 'Amaravati', 'Amarkantak', 'Amarpur', 'Amarpātan', 'Amarwāra', 'Ambad', 'Ambikāpur', 'Ambur', 'Ambāgarh Chauki', 'Ambāh', 'Ambājogāi', 'Ambāla', 'Amet', 'Amethi', 'Amethī', 'Amguri', 'Amini', 'Amla', 'Amloh', 'Ammāpettai', 'Amod', 'Ampati', 'Amreli', 'Amritsar', 'Amroha', 'Amrāvati', 'Amānganj', 'Amānpur', 'Anakāpalle', 'Anamalais', 'Anand', 'Anandnagar', 'Anandpur', 'Anantapur', 'Anantnag', 'Andol', 'Andrott', 'Anekal', 'Angamāli',